Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [14]:
train_subset = 10000
linear_graph = tf.Graph()
with linear_graph.as_default():
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [16]:

def evaluate(graph, num_steps):
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print ("Initialized")
        for step in range(num_steps):
            _, l, predictions = session.run([optimizer, loss, train_prediction])
            if (step % 100 == 0):
              print('Loss at step %d: %f' % (step, l))
              print('Training accuracy: %.1f%%' % accuracy(
                predictions, train_labels[:train_subset, :]))
              # Calling .eval() on valid_prediction is basically like calling run(), but
              # just to get that one numpy array. Note that it recomputes all its graph
              # dependencies.
              print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))  
        print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
evaluate(linear_graph, 801)

Initialized
Loss at step 0: 16.605181
Training accuracy: 8.9%
Validation accuracy: 12.9%
Loss at step 100: 2.249678
Training accuracy: 72.1%
Validation accuracy: 71.5%
Loss at step 200: 1.821841
Training accuracy: 74.8%
Validation accuracy: 73.5%
Loss at step 300: 1.589858
Training accuracy: 76.0%
Validation accuracy: 74.5%
Loss at step 400: 1.433209
Training accuracy: 76.7%
Validation accuracy: 75.0%
Loss at step 500: 1.316170
Training accuracy: 77.4%
Validation accuracy: 75.1%
Loss at step 600: 1.224061
Training accuracy: 77.9%
Validation accuracy: 75.3%
Loss at step 700: 1.149089
Training accuracy: 78.4%
Validation accuracy: 75.5%
Loss at step 800: 1.086550
Training accuracy: 78.8%
Validation accuracy: 75.6%
Test accuracy: 83.0%


In [24]:
train_subset = 10000
beta = 0.01
linear_l2_graph = tf.Graph()
with linear_l2_graph.as_default():
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * tf.nn.l2_loss(weights)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [25]:
evaluate(linear_l2_graph, 801)

Initialized
Loss at step 0: 50.623306
Training accuracy: 8.2%
Validation accuracy: 9.1%
Loss at step 100: 11.945908
Training accuracy: 72.9%
Validation accuracy: 72.1%
Loss at step 200: 4.518503
Training accuracy: 78.5%
Validation accuracy: 77.4%
Loss at step 300: 1.993073
Training accuracy: 81.4%
Validation accuracy: 80.3%
Loss at step 400: 1.136463
Training accuracy: 83.0%
Validation accuracy: 81.7%
Loss at step 500: 0.842908
Training accuracy: 83.7%
Validation accuracy: 82.0%
Loss at step 600: 0.741084
Training accuracy: 83.8%
Validation accuracy: 82.2%
Loss at step 700: 0.705433
Training accuracy: 83.8%
Validation accuracy: 82.3%
Loss at step 800: 0.692862
Training accuracy: 83.8%
Validation accuracy: 82.4%
Test accuracy: 88.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [26]:
train_subset = 1000
overfitting_graph = tf.Graph()
with overfitting_graph.as_default():
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [27]:
# Training accuracy will be very high, but test/validation accuracy will be lower.
evaluate(overfitting_graph, 801)

Initialized
Loss at step 0: 19.385155
Training accuracy: 7.1%
Validation accuracy: 10.3%
Loss at step 100: 1.360297
Training accuracy: 78.7%
Validation accuracy: 68.2%
Loss at step 200: 0.741848
Training accuracy: 86.8%
Validation accuracy: 69.7%
Loss at step 300: 0.472054
Training accuracy: 91.3%
Validation accuracy: 69.6%
Loss at step 400: 0.320857
Training accuracy: 94.4%
Validation accuracy: 69.6%
Loss at step 500: 0.228557
Training accuracy: 96.6%
Validation accuracy: 69.6%
Loss at step 600: 0.166571
Training accuracy: 97.4%
Validation accuracy: 69.7%
Loss at step 700: 0.124899
Training accuracy: 98.4%
Validation accuracy: 70.0%
Loss at step 800: 0.098479
Training accuracy: 99.1%
Validation accuracy: 70.3%
Test accuracy: 77.2%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [41]:
train_subset = 10000
batch_size = 128
num_hidden_nodes = 1024
dropout_probablity = 0.85

nn_graph = tf.Graph()
with nn_graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
    biases1 = tf.Variable(tf.zeros([num_hidden_nodes]))
    hidden_output = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), dropout_probablity)
    
    weights2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    logits= tf.matmul(hidden_output, weights2) + biases2
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction = tf.nn.softmax(logits)
    valid_hidden_layer_output = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_prediction = tf.nn.softmax(
        tf.matmul(valid_hidden_layer_output, weights2) + biases2)
    test_hidden_layer_output = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(
        tf.matmul(test_hidden_layer_output, weights2) + biases2)

In [42]:
num_steps = 3001

with tf.Session(graph=nn_graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 315.124329
Minibatch accuracy: 14.1%
Validation accuracy: 36.6%
Minibatch loss at step 500: 19.276627
Minibatch accuracy: 79.7%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 8.642092
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 6.921552
Minibatch accuracy: 82.8%
Validation accuracy: 80.6%
Minibatch loss at step 2000: 13.731734
Minibatch accuracy: 75.8%
Validation accuracy: 78.3%
Minibatch loss at step 2500: 7.871657
Minibatch accuracy: 78.9%
Validation accuracy: 82.1%
Minibatch loss at step 3000: 4.677133
Minibatch accuracy: 80.5%
Validation accuracy: 81.1%
Test accuracy: 88.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
